In [2]:
from tensorflow import keras

### 데이터 읽기

In [47]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [48]:
train_input.shape, test_input.shape, train_target.shape, test_target.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

### 데이터 정규화

In [50]:
# 산술식으로 정규화하기
train_scaled = train_input / 255.0
train_scaled.shape

(60000, 28, 28)

### 픽셀 데이터를 1차원으로 변환

In [52]:
train_scaled = train_scaled.reshape(-1, 28 * 28)
train_scaled.shape

(60000, 784)

### 훈련데이터와 검증데이터 분리

In [54]:
train_scaled.shape, train_target.shape

((60000, 784), (60000,))

In [55]:
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
# train_scaled.shape, val_scaled.shape, train_target.shape, val_target.shape

### 입력계층

In [76]:

# 784개의 입력데이터를 이용해서 손실이 적은 100개 추출
dense1 = keras.layers.Dense(100, activation='sigmoid',input_shape=(784,))


### 출력계층

In [57]:

# 입력계층을 통해 추출된 100개의 데이터를 입력으로 박아서 최종 10개로 분류(확률적으로 분류됨)
# 출력계층의 활성화 함수 : sigmoid, softmax 둘중 하나
dense2 = keras.layers.Dense(10,activation='softmax')

### 심층신경망 만들기(신경망층이 여러개인 경우)
### 모델생성

In [77]:
model = keras.Sequential([dense1, dense2])
model

### 모델구조확인

In [60]:

model.summary()

# dense1 파라키터 갯수 = 784 * 100 + 100 = 78500
# dense2 파라키터 갯수 = 100 * 10 + 10 = 1010

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


###  신경망 계층 추가방법(2)
#### - 신경망모델 생성시에 신경망층을 함께 생성

In [61]:
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden_layer'),
                          keras.layers.Dense(10, activation='softmax', name='out_layer')
                         ], name='Model')
model


In [62]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer (Dense)        (None, 100)               78500     
                                                                 
 out_layer (Dense)           (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


###  신경망 계층 추가방법(3)
#### - 신경망모델 생성 후 add()함수를 이용해서 계층 추가
#### - 가장 많이 사용

In [63]:
# 1.모델 생성
model = keras.Sequential()
model

In [64]:
# 2.계층 추가
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [65]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [66]:
# 3.모델 설정(compile)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

In [67]:
# 4.모델 훈련
model.fit(train_scaled, train_target, epochs=10)

Epoch 1/10
1500/1500 [==============================] - 1s 565us/step - loss: 0.5665 - accuracy: 0.8059
Epoch 2/10
1500/1500 [==============================] - 1s 543us/step - loss: 0.4064 - accuracy: 0.8527
Epoch 3/10
1500/1500 [==============================] - 1s 556us/step - loss: 0.3707 - accuracy: 0.8653
Epoch 4/10
1500/1500 [==============================] - 1s 541us/step - loss: 0.3483 - accuracy: 0.8734
Epoch 5/10
1500/1500 [==============================] - 1s 546us/step - loss: 0.3313 - accuracy: 0.8806
Epoch 6/10
1500/1500 [==============================] - 1s 540us/step - loss: 0.3175 - accuracy: 0.8839
Epoch 7/10
1500/1500 [==============================] - 1s 550us/step - loss: 0.3058 - accuracy: 0.8889
Epoch 8/10
1500/1500 [==============================] - 1s 560us/step - loss: 0.2979 - accuracy: 0.8905
Epoch 9/10
1500/1500 [==============================] - 1s 555us/step - loss: 0.2884 - accuracy: 0.8960
Epoch 10/10
1500/1500 [==============================] - 1s 546u

In [70]:
# 5.검증하기
scores = model.evaluate(val_scaled, val_target)
scores

375/375 [==============================] - 0s 384us/step - loss: 0.3235 - accuracy: 0.8848


[0.3234919607639313, 0.8848333358764648]

In [75]:
print('손실율 : ',scores[0], '정확도 :', scores[1])

손실율 :  0.3234919607639313 정확도 : 0.8848333358764648


### 손실함수(loss function)
#### 손실함수는 실제값과 예측값의 차이를 수치화해주는 함수
#### 이 두값의 차이, 즉 오차가 클수록 손실함수의 값은 크고 오차가 작을수록 손실함수의 값은 작음
#### "회귀"와 같이 연속 데이터를 사용하는 경우에는 "평균제곱오차"를 사용
#### "분류"에서는 크로스 엔트로피를 주로 사용
#### 딥러닝의 학습과정은 손실함수의 값이 최소화되는 지점의 가중치와 편향을 찾는것이 목적임

#### < 손실 함수로 사용되는 함수들 >
#### 1. MSE(Mean Squared Error): 평균제곱오차 - 연속현변수를 예측할 때 사용
####  - compile()함수에서 loss속성에 "mse"라고 작성하여 사용

#### 2.Categorical Crossentropy
####  - 딥러닝에서 다중 분류시에 사용
####  - target 데이터의 형태가 원핫인코딩된 형태인 경우 사용

#### 3. Binary Crossentropy
####  - target 데이터의 분류형태가 0 또는 1과 같은 이진분류시 사용
####  - 출력층의 활성화함수로 시그모이드(sigmoid)함수를 사용

#### 4.Sparse Categorical Crossentropy
####  - target 데이터의 형태가 0,1,2,...와 같은 숫자형태인 경우 사용
####  - 출력층의 활성화 함수로는 소프트맥스(softmax)함수를 사용

### 주로 3,4번을 많이 사용
---
### <활성화 함수 Activation Function>
#### - 손실에 따른 풀력을 담당하는 함수로 종속변수(데이터)를 결정하기 위한 함수
#### - 중간 단계의 게층(은닉층)에서는 출력 개수에 맞는 결과를 결정하기 위해 활성화함수가 사용되며
#### - 출력단계의 계층(출력층)에서는 최종 target 데이터를 결정하기 위한 활성화함수로 사용됨


#### < 사용되는 활성화함수 >
#### 1. Sigmoid
####  - logistic function이라고도 함
####  - 시그모이드 함수의 결과값이 0과 1사이의 값으로 결정됨
####  - y의 값이 0.5를 기준으로 05보다 크면 1, 작으면 0으로 결정됨
####  - 이진분류시에 주로 사용됨
####  - 손실함수는 Binary Crossentropy 사용

#### 2. Than(Hypterbolic Tangent)
####  - 1에서 -1사이의 값으로 결정됨
####  - 0보다 크면 1, 작으면 -1로 결정
####  - 시그모니드보다 결정범위가 크기에 학습이 효율적이라고 평함

#### 3.Relu(Rectified Linear Unit)
####  - 0과 1의 값으로 처리
####  - 0보다 작으면 모두 0, 0보다 크면 1로 처리함
####  - 0보다 작은 값이 발생하지 않는다는 장점이 있음
####  - 중간계층에 주로 사용

#### 4. Softmax
####  - 0.0에서 1.0사이의 실수의 범위로 사용
####  - 여러개의 target값중에 가장 높은 결정값을 선택하게 되며 여러개의 target값 들은 0.1~1.0사이의 확률로 나타남
####  - target 값의 모든 값의 합은 1.0이 됨
####  - 출력층에 주로 사용
####  - 손실계수로는 Sparse Categorical Crossentropy가 사용됨

#### 5.Leaky Relu
#### -Relu의 향상됨 모델로 0보다 작은 값들을 허용하게 하여 훈련의 효율을  높였음
---

### 딥러닝 계층(layer)을 사용하는 방법
#### 1. 다른 누군가가 만들어 놓은 모델을 그대로 사용
#### 2. 계층을 추가 또는 제거하면서
#### 3. 사용하는 데이터에 가장 적합한(정확도가 높은) 모델을 완성

---
### 전처리 계층(layer)사용하기


In [78]:
# 모델생성

In [86]:
model = keras.Sequential()

### 전처리계층 추가하기
#### 훈련에 사용되지 않음
####  - Flatten()
####  - 1차원으로 만들어주는 계층
####  - 입력차원을 모두 일렬로 펼치는 역할 수행
####  - 입력에 곱해지는 가중치나 절편이 없음
####  - 인공신경망의 성능에 기여하지 않음(훈련에 사용되지 않음의 의미)
####  - 입력층과 은닉층 사이에 위치함
####  - 보통 입력층 바로 뒤에 위치하는 것이 일반적임

In [88]:
model.add(keras.layers.Flatten(input_shape=(28,28)))

In [ ]:
### 은닉계층(hidden layer) 추가하기

In [89]:
model.add(keras.layers.Dense(100, activation='relu'))

In [ ]:
### 출력계층(output layer) 추가하기

In [91]:
model.add(keras.layers.Dense(10, activation='softmax'))

In [96]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 100)               78500     
                                                                 
 dense_10 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


### 실습하기

In [93]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_input.shape, test_input.shape, train_target.shape, test_target.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [94]:
train_scaled = train_input / 255.0
train_scaled.shape

(60000, 28, 28)

In [95]:
from sklearn.model_selection import train_test_split
train_scaled, val_sclaed, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)
train_scaled.shape, val_sclaed.shape, train_target.shape, val_target.shape

((48000, 28, 28), (12000, 28, 28), (48000,), (12000,))

### 컴파일 : 훈련모델 손실함수 설정

In [97]:
model.compile(loss='sparse_categorical_crossentropy',
             metrics='accuracy')

### 훈련하기

In [98]:
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 551us/step - loss: 0.5329 - accuracy: 0.8126
Epoch 2/5
1500/1500 [==============================] - 1s 554us/step - loss: 0.3903 - accuracy: 0.8603
Epoch 3/5
1500/1500 [==============================] - 1s 543us/step - loss: 0.3548 - accuracy: 0.8727
Epoch 4/5
1500/1500 [==============================] - 1s 531us/step - loss: 0.3353 - accuracy: 0.8796
Epoch 5/5
1500/1500 [==============================] - 1s 538us/step - loss: 0.3191 - accuracy: 0.8854


###  검증하기

In [100]:
scores = model.evaluate(val_sclaed, val_target)
scores

375/375 [==============================] - 0s 534us/step - loss: 0.3517 - accuracy: 0.8802


[0.3516687750816345, 0.8801666498184204]

###  옵티마이저를 이용한 모델 클래스 설정
#### < 옵티마이저 - Optimizer >
#### - 딥러닝에서 빠르고 정확하게 학습하는 것을 목표로 나온 속성
#### - 주로 경사하강법(Gradient Descent Algorithm)기반의 알고리즘을 지정
#### - 경사하강법의 옵티마이저 : SGD기반으로 딥러닝 모델들이 향상되고 있음
#### - 다양한 경사하강법 알고리즘이 있음
#### - compile()함수에 설정함
---
#### 학습율(learning_rate)
#### - 기본값은 0.01, 값이 작을수록 훈련시간이 오래 걸림
#### - 값이 커질수록 훈련강도가 낮아지게 됨
#### - 가중치를 최적화하는데 있어서 학습율의 값은 매우 중요하게 작용됨
#### - 학습율의 값은 하이퍼파라미터 튜닝을 통해 찾아내며 작성할수도 있음
#### - 학습율은 경사를 내려오면서 얼마만큼 이동할 것인지를 결정하는 것임(예: 사람의 걷는 보폭)
---
### 모멘텀(momentum)
#### - 학습율과 동일하게 최적화 알고리즘임
#### - 물리학에서 사용하는 운동량과 동력을 뜻함
#### - 랜덤하게 지그재그로 내려오면서 최적화(최단경사 이동거리)되는 것을 해결하기 위해 나온 알고리즘
#### - ** 관성과 가속도를 적용한 알고리즘 ** -
#### - 이동하던 한 방향으로 좀 더 이동할 수 있도록 하기 위해 적용된 개념

#### < 모멘텀 주요 특징>
#### - 현재 이동하는 방향과는 별개로 과거에 이동했던 방향을 기억하고 있음
#### - 과거의 방향으로 일정량(학습율)을 추가하여 이동을 하게 됨
#### - 기존 데이터를 기억하고 있어야 하므로 메모리 사용량이 많아지는 단점이 있음

####  nesterov(네스테로프)
#### -모멘텀 최적화의 변종 알고리즘
#### - 현재 위치가 아니라 모멘텀의 조금 미리 앞서서 경사를 계산
---

### 적응적 학습율(adaptive learning rate) - 학습율을 자동 조정
#### - 모델이 최적점(목표하는 지점)에 다달았을 때 학습률(보폭)을 낮추게 되면 안정적으로 최적점에 도달(수렴)할 것이라는 가능성이 높아진다는 개념에서 나온 알고리즘
#### - 케라스 모델이 자동으로 학습율을 조정하게 됨
#### - 학습율을 자동으로 조정하는 것을 "조정적 학습율"이라고 함  - 사람이 직접 처리해 주지 않아도 됨
#### - 적응적 학습율의 기본 학습율은 0.01임
#### - 적응적 학습율 알고리즘 : adagrad, rmsprop(기본 학습법), adam 등
---
###

In [107]:
# 경사하강법을 적용
model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
             metrics='accuracy')

# 위와 동일
sgd = keras.optimizers.SGD(learning_rate=0.1)

# 모멘텀 학습율
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

adagrad = keras.optimizers.Adagrad()

rmsprop = keras.optimizers.RMSprop()

adam = keras.optimizers.Adam()

model.compile(optimizer=sgd,
             loss='sparse_categorical_crossentropy',
             metrics='accuracy')

model.compile(optimizer=adagrad,
             loss='sparse_categorical_crossentropy',
             metrics='accuracy')

model.compile(optimizer=rmsprop,
             loss='sparse_categorical_crossentropy',
             metrics='accuracy')

model.compile(optimizer=adam,
             loss='sparse_categorical_crossentropy',
             metrics='accuracy')

###  인공신경망 모델 생성
#### 1.모델 생성

In [113]:
model = keras.Sequential()


#### 2. 전처리 계층

In [114]:
model.add(keras.layers.Flatten(input_shape=(28,28)))

#### 3.은닉층 추가

In [115]:
model.add(keras.layers.Dense(100, activation='relu'))

####  4.출력층 추가

In [116]:
model.add(keras.layers.Dense(10,activation='softmax'))

####  5. 모델구조 확인

In [112]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               78500     
                                                                 
 dense_12 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


#### 6.컴파일
#### - adam 적응적 학습율을 적용
#### - adam : 방향과 학습율 두가지 모두를 적절하게 처리하기 위한 알고리즘
#### - 옵티마이저로 가장 먼저 적용하는것이 좋음

In [118]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics='accuracy')

#### 7.모델훈련

In [119]:
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 481us/step - loss: 0.5205 - accuracy: 0.8167
Epoch 2/5
1500/1500 [==============================] - 1s 488us/step - loss: 0.3918 - accuracy: 0.8598
Epoch 3/5
1500/1500 [==============================] - 1s 469us/step - loss: 0.3519 - accuracy: 0.8720
Epoch 4/5
1500/1500 [==============================] - 1s 477us/step - loss: 0.3253 - accuracy: 0.8818
Epoch 5/5
1500/1500 [==============================] - 1s 480us/step - loss: 0.3056 - accuracy: 0.8886


#### 8.모델검증

In [121]:
scores = model.evaluate(val_sclaed, val_target)
scores

375/375 [==============================] - 0s 384us/step - loss: 0.3739 - accuracy: 0.8670


[0.3739257752895355, 0.8669999837875366]

#### 옵티마이저 3가지를 처리하면서 최저손실율과 최대 정확도를 찾아보기

In [126]:
def getbesteval(opt, epoch):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(10,activation='softmax'))
    model.compile(optimizer=opt,
                 loss='sparse_categorical_crossentropy',
                 metrics='accuracy')
    model.fit(train_scaled, train_target, epochs=epoch)
    rs_eval = model.evaluate(val_sclaed, val_target)
    return rs_eval

In [127]:
optimizer = ['adagrad','rmsprop','adam']
best_eval = 0.0
best_opt = ""

best_loss = 1.0
best_loss_opt =""

for opt in optimizer:
    rs = getbesteval(opt,2)
    print("opt=", opt, "| rs =",rs)
    
    # 가장 낮은 손실율과 이때 옵티마이저 확인하기
    if best_loss > rs[0]:
        best_loss = rs[0]
        best_loss_opt = opt
        print("best_loss_opt =", best_loss_opt,
             "| best_loss = ",best_loss,
             "| rs[0] = ", rs[0])

    # 가장 높은 손실율과 이때 옵티마이저 확인하기
    if best_loss > rs[1]:
        best_loss = rs[1]
        best_loss_opt = opt
        print("best_loss_opt =", best_loss_opt,
             "| best_loss = ",best_loss,
             "| rs[1] = ", rs[1])  
        
print(best_opt, best_eval,"/",best_loss_opt, best_loss)

Epoch 1/2
1500/1500 [==============================] - 1s 487us/step - loss: 1.1690 - accuracy: 0.6516
Epoch 2/2
375/375 [==============================] - 0s 387us/step - loss: 0.7408 - accuracy: 0.7580
opt= adagrad | rs = [0.740838885307312, 0.7580000162124634]
best_loss_opt = adagrad | best_loss =  0.740838885307312 | rs[0] =  0.740838885307312
Epoch 1/2
1500/1500 [==============================] - 1s 541us/step - loss: 0.5344 - accuracy: 0.8104
Epoch 2/2
375/375 [==============================] - 0s 390us/step - loss: 0.3645 - accuracy: 0.8706
opt= rmsprop | rs = [0.3645346760749817, 0.8705833554267883]
best_loss_opt = rmsprop | best_loss =  0.3645346760749817 | rs[0] =  0.3645346760749817
Epoch 1/2
1500/1500 [==============================] - 1s 478us/step - loss: 0.5271 - accuracy: 0.8161
Epoch 2/2
375/375 [==============================] - 0s 376us/step - loss: 0.3846 - accuracy: 0.8618
opt= adam | rs = [0.3845868706703186, 0.8618333339691162]
 0.0 / rmsprop 0.3645346760749817
